In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
from glob import glob
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model, regularizers
from tensorflow.keras.layers import Input, Dense, Dropout, Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
import cv2 as cv
import matplotlib.pyplot as plt

In [ ]:
#os.listdir('/kaggle/input/pokemon-images-and-types/images/images')

In [ ]:
#os.listdir('/kaggle/input/pokemon-images-and-types/')

# Images Preprocessing

In [ ]:
def img_preprocess():
    img_list = []

    for img_path in glob('/kaggle/input/pokemon-images-and-types/images/images/*.png'):
        img = cv.imread(img_path)            # cv 讀進來後是 3-dim 陣列，last_channel = 3，代表色彩三元素RGB
        img = cv.resize(img, (224, 224))     # 將圖片 shape 轉換為 (224, 224)
        sc_img = img.astype('float') / 255   # 標準化（Normalization）
        img_list.append(sc_img)
    
    return np.asarray(img_list)

In [ ]:
img_arr = img_preprocess()

In [ ]:
plt.imshow(img_arr[0])
plt.xticks([])
plt.yticks([])

In [ ]:
img_arr.shape

# Build Model

# Convolutional AutoEncoder

# -使用 UpSampling2D 作反卷積

In [ ]:
input_layer = Input(shape=(224, 224, 3))

x = Conv2D(filters=64,
           kernel_size=(3,3),
           padding='same', 
           activation='relu')(input_layer)
x = MaxPooling2D(pool_size=(2,2),
                 strides=(2,2))(x)
x = Conv2D(filters=32,
           kernel_size=(3,3),
           padding='same',
           activation='relu')(x)
x = MaxPooling2D(pool_size=(2,2),
                 strides=(2,2))(x)
x = Conv2D(filters=32,
           kernel_size=(3,3),
           padding='same',
           activation='relu')(x)
encoder_layer = MaxPooling2D(pool_size=(2,2),
                             strides=(2,2))(x)

x = Conv2D(filters=32,
           kernel_size=(3,3),
           padding='same',
           activation='relu')(encoder_layer)
x = UpSampling2D(size=(2,2))(x)              # UpSampling2D 功用為將傳統卷積 Down Sample 後的向量再進行放大（還原），亦可稱「反卷積」。
x = Conv2D(filters=32,
           kernel_size=(3,3),
           padding='same',
           activation='relu')(x)
x = UpSampling2D(size=(2,2))(x)
x = Conv2D(filters=64,
           kernel_size=(3,3),
           padding='same',
           activation='relu')(x)
x = UpSampling2D(size=(2,2))(x)
decoder_layer = Conv2D(filters=3,
                       kernel_size=(3,3),
                       padding='same',
                       activation='sigmoid')(x)

#-- 建立 Convolutional AutoEncoder --#
conv_auto_encoder = Model(input_layer, decoder_layer)

#-- 建立 Encoder --#
encoder = Model(input_layer, encoder_layer)

In [ ]:
conv_auto_encoder.summary()

In [ ]:
conv_auto_encoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
callbacks = EarlyStopping(monitor='val_loss',
                          patience=11,
                          restore_best_weights=True)

In [ ]:
history = conv_auto_encoder.fit(img_arr, img_arr,
                                epochs=300, batch_size=64,
                                validation_split=0.3,
                                shuffle=True,
                                callbacks=[callbacks])

In [ ]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.show()

In [ ]:
encoded = encoder.predict(img_arr)
decoded = conv_auto_encoder.predict(img_arr)

In [ ]:
print ('Low dimention shape: {}'.format(encoded[0].shape))
print ('output dimention shape: {}'.format(decoded[0].shape))

In [ ]:
fig = plt.figure(figsize=(28, 12))

n_plots = 10
n_row = int(n_plots / 2)

for j in range(n_plots):
    #-- 原始輸入值（Input） --#
    fig.add_subplot(n_row, 6, 3*j+1)
    plot_tmp = plt.imshow(img_arr[j])
    plt.xticks([])
    plt.yticks([])
    
    #-- 低維陣列 --#
    fig.add_subplot(n_row, 6, 3*j+2)
    plot_tmp = plt.imshow(encoded[j].reshape(32, 784))
    plt.xticks([])
    plt.yticks([])
    
    #-- Decoder 重建（Reconstruct）原始資料 --#
    fig.add_subplot(n_row, 6, 3*j+3)
    plot_tmp = plt.imshow(decoded[j])
    plt.xticks([])
    plt.yticks([])
    
plt.show()

# -使用 Conv2DTranspose 作反卷積

In [ ]:
input_layer = Input(shape=(img_arr.shape[1], img_arr.shape[2], img_arr.shape[3]))

x = Conv2D(filters=64,
           kernel_size=(3,3),
           padding='same',
           activation='relu')(input_layer)
x = MaxPooling2D(pool_size=(2,2), strides=(2,2))(x)

x = Conv2D(filters=32,
           kernel_size=(3,3),
           padding='same',
           activation='relu')(x)
x = MaxPooling2D(pool_size=(2,2), strides=(2,2))(x)

x = Conv2D(filters=32,
           kernel_size=(3,3),
           padding='same',
           activation='relu')(x)
encoder_layer = MaxPooling2D(pool_size=(2,2), strides=(2,2))(x)

x = Conv2D(filters=32,
           kernel_size=(3,3),
           padding='same',
           activation='relu')(encoder_layer)
x = Conv2DTranspose(filters=32,
                    kernel_size=(3,3),
                    strides=(2,2),
                    padding='same',
                    use_bias=True,
                    activation='relu')(x)
x = Conv2D(filters=32,
           kernel_size=(3,3),
           padding='same',
           activation='relu')(x)
x = Conv2DTranspose(filters=32,
                    kernel_size=(3,3),
                    strides=(2,2),
                    padding='same',
                    use_bias=True,
                    activation='relu')(x)
x = Conv2D(filters=64,
           kernel_size=(3,3),
           padding='same',
           activation='relu')(x)
x = Conv2DTranspose(filters=64,
                    kernel_size=(3,3),
                    strides=(2,2),
                    padding='same',
                    use_bias=True,
                    activation='relu')(x)
decoder_layer = Conv2D(filters=3,
                       kernel_size=(3,3),
                       padding='same',
                       activation='sigmoid')(x)

#-- 建立 Convolutional AutoEncoder --#
conv_auto_encoder = Model(input_layer, decoder_layer)

#-- 建立 Encoder --#
encoder = Model(input_layer, encoder_layer)

In [ ]:
conv_auto_encoder.summary()

In [ ]:
conv_auto_encoder.compile(optimizer='adam', loss='binary_crossentropy')

In [ ]:
callbacks = EarlyStopping(monitor='val_loss',
                          patience=11,
                          restore_best_weights=True)

In [ ]:
history = conv_auto_encoder.fit(img_arr, img_arr,
                                epochs=300, batch_size=64,
                                validation_split=0.3,
                                shuffle=True,
                                callbacks=[callbacks])

In [ ]:
encoded = encoder.predict(img_arr)
decoded = conv_auto_encoder.predict(img_arr)

In [ ]:
print ('encoded img shape {}'.format(encoded.shape))
print ('decoded img shape {}'.format(decoded.shape))

In [ ]:
fig = plt.figure(figsize=(28, 12))

n_plot = 10
n_rows = int(n_plot / 2)

for j in range(n_plot):
    fig.add_subplot(n_rows, 6, 3*j+1)
    plot_tmp = plt.imshow(img_arr[j])
    plt.xticks([])
    plt.yticks([])
    
    fig.add_subplot(n_rows, 6, 3*j+2)
    plot_tmp = plt.imshow(encoded[j].reshape(32, 784))
    plt.xticks([])
    plt.yticks([])
    
    fig.add_subplot(n_rows, 6, 3*j+3)
    plot_tmp = plt.imshow(decoded[j])
    plt.xticks([])
    plt.yticks([])